[View in Colaboratory](https://colab.research.google.com/github/suzuki-tr/tensorflow-study/blob/master/TensorFlow_with_GPU.ipynb)

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
tf.__version__

'1.6.0'

# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [2]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
8.436719796999967
GPU (s):
0.8491997469999433
GPU speedup over CPU: 9x


# Create Multiple GRAPH and Use one of them in SESSION.

In [75]:
import tensorflow as tf

# Create multiple graph
graph_1 = tf.Graph()
with graph_1.as_default():
  x = tf.placeholder(tf.float32, shape=[3],name='Input')
  y = tf.square(x,name='Middle')
  z = tf.square(y,name='Output')
  print('graph_1',graph_1)

graph_2 = tf.Graph()
with graph_2.as_default():
  x = tf.placeholder(tf.float32, shape=[3],name='Input')
  c = tf.constant([1.0,1.0,1.0])
  y = tf.add(x,c,name='Middle')
  z = tf.add(y,c, name='Output')
  print('graph_2',graph_2)

# Use each graph
with graph_1.as_default():
  print('graph_1',tf.get_default_graph())
  with tf.Session() as sess:
    for op in sess.graph.get_operations():
      print (op.name)
    print (sess.run('Output:0', {'Input:0': [1.0, 2.0, 3.0]}))
    print (sess.run('Output:0', {'Middle:0': [1.0, 2.0, 3.0]}))
    print (sess.run('Middle:0', {'Input:0': [1.0, 2.0, 3.0]}))

    
with graph_2.as_default() as g:
  print('graph_2',tf.get_default_graph())
  with tf.Session() as sess:
    for op in sess.graph.get_operations():
      print (op.name)
    print (sess.run('Output:0', {'Input:0': [1.0, 2.0, 3.0]}))
    print (sess.run('Output:0', {'Middle:0': [1.0, 2.0, 3.0]}))
    print (sess.run('Middle:0', {'Input:0': [1.0, 2.0, 3.0]}))


graph_1 <tensorflow.python.framework.ops.Graph object at 0x7f3a70eb2cc0>
graph_2 <tensorflow.python.framework.ops.Graph object at 0x7f3a70eb2f98>
graph_1 <tensorflow.python.framework.ops.Graph object at 0x7f3a70eb2cc0>
Input
Middle
Output
[ 1. 16. 81.]
[1. 4. 9.]
[1. 4. 9.]
graph_2 <tensorflow.python.framework.ops.Graph object at 0x7f3a70eb2f98>
Input
Const
Middle
Output
[3. 4. 5.]
[2. 3. 4.]
[2. 3. 4.]


In [47]:
graph = tf.Graph()
with graph.as_default() as g:
 
  c_0 = tf.constant(0, name="c")  # => operation named "c"

  # Already-used names will be "uniquified".
  c_1 = tf.constant(2, name="c")  # => operation named "c_1"

  # Name scopes add a prefix to all operations created in the same context.
  with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

  # Name scopes nest like paths in a hierarchical file system.
  with tf.name_scope("inner"):
    c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

  # Exiting a name scope context will return to the previous prefix.
  c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

  # Already-used name scopes will be "uniquified".
  with tf.name_scope("inner"):
    c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"
  
  for op in g.get_operations():
    print(op.name)

c
c_1
outer/c
inner/c
c_2
inner_1/c


In [57]:
graph = tf.Graph()
with graph.as_default() as g:
  x = tf.constant([[1.0, 2.0], [1.0, 2.0]])
  w = tf.constant([[1.0, 1.0], [1.0, 1.0]])
  y = tf.matmul(x, w)
  output = tf.nn.softmax(y)

  with tf.Session() as sess:
    print(sess.run(x))
    print(sess.run(w))
    print(sess.run(output))
    print(sess.run([y, output]))
    #print(y_val, output_val)

[[1. 2.]
 [1. 2.]]
[[1. 1.]
 [1. 1.]]
[[0.5 0.5]
 [0.5 0.5]]
[array([[3., 3.],
       [3., 3.]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5]], dtype=float32)]
